An `Interchange` is most commonly constructed via the `Interchange.from_smirnoff()` class method. This method takes a SMIRNOFF force field and applies it to a molecular topology. The [OpenFF Toolkit](https://docs.openforcefield.org/projects/toolkit/en/stable/) provides these objects - [`ForceField`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.typing.engines.smirnoff.ForceField.html#openff.toolkit.typing.engines.smirnoff.ForceField) and [`Topology`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology), respectively.

Other input pathways are in development and will be discussed later. But the key interoperability pathway depends on the prior preparation of these objects. Equivalently, force fields, molecules, and topologies that cannot be coerced into these formats are not supported by `Interchange.from_smirnoff()`.

In [ ]:
from openff.interchange import Interchange

Interchange.from_smirnoff?

Internally, the toolkit's SMARTS-matching algorithms are used to assign parameters to the chemical topology. Recall that SMIRNOFF's use of [direct chemical perception](https://pubs.acs.org/doi/10.1021/acs.jctc.8b00640) bypasses the use of atom types, so atom types to not exist as first-class objects in `Interchange` objects.

Preparing the force field and topology inputs is outside the scope of Interchange _per se_, but some time will be spent on system preparation.

`Molecule` objects in the OpenFF Toolkit are rich descrptions of chemistry akin to cheminformatics representations  found in RDKit and OpenEye Toolkits. This is distinct from most molecule mechanics use cases in which coordinates, elements, and bonds are sufficient. The preferred starting point for small molecules is something like an SDF file or a SMILES pattern. (Later, we'll look at how the toolkit can load proteins from PDB files.) For starters, let's generate an ethanol molecule and corresponding topology.

In [ ]:
from openff.toolkit import Molecule, Topology

molecule = Molecule.from_smiles("c1ccc(cc1)C[C@@H](C(=O)O)N")
molecule.generate_conformers(n_conformers=2)
molecule

In [ ]:
topology = Topology.from_molecules([molecule])

Loading a force field is, by comparison, straightforward. Let's use OpenFF 2.0.0 (code name "Sage"), the latest offering from the Open Force Field Initiative.

In [ ]:
from openff.toolkit import ForceField

sage = ForceField("openff-2.0.0.offxml")

Interchange implements all of the current SMIRNOFF specification (except for GBSA models). This makes it plug-and-play for some features that are not yet in mainline OpenFF force fields - namely virtual sites and WBO-based parameter interpolation. Interchange is already compatible with the contents of a SMIRNOFF force field for biomolecules, so once one is released in the form of Rosemary, it should be a drop-in replacement for Sage. If you re-fit torsions in a specific ligand and create a bespoke force field using BespokeFit, this can immediately be loaded it an used with Interchange.

Now we have the two required inputs for creating an `Interchange`, so let's make one.

In [ ]:
interchange = Interchange.from_smirnoff(
    force_field=sage,
    topology=topology,
)
interchange

Note that, for convience and familiarity compared to the existing `ForceField.create_openmm_system` method, the toolkit provides `ForceField.create_interchange`, which effectively wraps `Interchange.from_smirnoff` to the same effect.

In [ ]:
interchange2 = sage.create_interchange(topology)
interchange2

This object stores all information known about a system; this includes its chemistry, how that chemistry is represented by a force field, and how the system is organized in 3D space. An Interchange object has five components:

1. **Topology**: Stores chemical information, such as connectivity and formal charges, independently of force field
1. **Handlers**: Maps the chemical information to force field parameters. The force field itself is not directly stored
1. **Positions** (optional): Cartesian co-ordinates of atoms
1. **Box vectors** (optional): Periodicity information
1. **Velocities** (optional): Cartesian velocities of atoms

None are strictly required; an `Interchange` object can be constructed containing none of the above components, although this is not particularly useful:


In [ ]:
empty = Interchange()
empty.topology, empty.handlers, empty.positions, empty.box, empty.velocities

The `Interchange.topology` attribute carries an object of the same type provided by the toolkit and therefore provides the same API. (In the future this may change).

In [ ]:
interchange.topology.n_atoms, interchange.topology.n_bonds, interchange.topology.molecule(
    0
).to_smiles()

The `Interchange.handlers` attribute carries a dictionary mapping handler names to `SMIRNOFFPotentialHandler` objects. These carry the physical parameters derived from applying the force field to the topology. We will go into more detail about these objects later.

In [ ]:
[(key, type(value)) for key, value in interchange.handlers.items()]

The `Interchange.positions` attribute carries the positions of all atoms if specified in the constructor or included on the provided topology. In our case, we passed a topology with a molecule that had a defined conformer, so `from_smirnoff` set atomic positions from this information.

In [ ]:
import numpy

assert numpy.allclose(interchange.positions, molecule.conformers[0])

type(interchange.positions), interchange.positions

`Interchange.positions` also has a setter that allows array-like objects of the shape `(N_atoms, 3)`. For example, we could switch this `interchange` to use the second of the two conformers we generated earlier:

In [ ]:
interchange.positions = molecule.conformers[1]

assert not numpy.allclose(interchange.positions, molecule.conformers[0])
assert numpy.allclose(interchange.positions, molecule.conformers[1])

type(interchange.positions), interchange.positions

The `Interchange.box` attribute carries information about the periodicity of the system. It can be `None` or a `Quantity` object, with the `None` implying a lack of periodicity. If no `box` argument is passed to `Interchange.from_smirnoff`, it will set box vectors, if any, from the `topology` input. After construction, it can be set with the same process with which we modified the positions.

In [ ]:
assert topology.box_vectors == interchange.box == None
from openff.units import unit

interchange.box = unit.Quantity(4.0 * numpy.eye(3), unit.nanometer)
interchange.box

The `Interchange.velocities` attribute carries atomic velocities, which can be set akin to setting other array attributes - by passing an `(N_atoms, 3)` array to the setter.